### This notebook contains my code for obtaining the parameters of the single Gaussian classifier. The training data was labeling in MatLab using the roipoly function. 

In [73]:
import numpy as np
import math
import cv2

#### Organize all pixels and corresponding labels into 3xN and 1xN where N is the total number of pixels or training samples. I use the last 41 images of the data set for training, and the first 5 as a validation set.

In [88]:
x_train = np.zeros((3,0))
y_train = np.zeros((1,0))

for i in range(5,45):
    img = cv2.imread("trainset/%d.png" %(i+1))
    array_img = np.array(img)    
    array_img = np.moveaxis(array_img, -1, 0)
    array_img = np.reshape(array_img, (3,-1))
    
    img_mask = Image.open("trainset/%dbb.png" %(i+1))
    array_mask = np.array(img_mask)
    array_mask = np.reshape(array_mask, (1,-1))
    
    x_train = np.append(x_train,array_img, axis = 1)
    y_train = np.append(y_train,array_mask)#, axis = 1)

N = x_train.shape[1]
print(x_train.shape)
print(y_train.shape)                


(3, 38400000)
(38400000,)


#### Separate training samples corresponding to blue and not blue.


In [89]:
x_train_blue = x_train[:, y_train == 1]
print(x_train_blue.shape)

x_train_not_blue = x_train[:, y_train == 0]
print(x_train_not_blue.shape)

N_blue = x_train_blue.shape[1]
N_not_blue = x_train_not_blue.shape[1]
print(N_blue + N_not_blue)

(3, 347713)
(3, 37914963)
38262676


#### Compute average of blue/not blue classes. This is the MLE estimate of the mean for each class.

In [90]:
blue_sum = np.sum(x_train_blue, 1, keepdims=True)
blue_average = blue_sum / x_train_blue.shape[1]
print(blue_average)

not_blue_sum = np.sum(x_train_not_blue, 1, keepdims=True)
not_blue_average = not_blue_sum / x_train_not_blue.shape[1]
print(not_blue_average)

[[115.00799798]
 [ 65.94341023]
 [ 32.35539942]]
[[ 87.11597537]
 [ 98.0918099 ]
 [104.67259641]]


#### Compute covariance of blue/not blue classes. This is the MLE estimate of the covariance for each class.

In [91]:
blue_minus_mean = x_train_blue - blue_average
print(blue_minus_mean.shape)

blue_cov = np.dot(blue_minus_mean, blue_minus_mean.T)/ N_blue
print(blue_cov)


not_blue_minus_mean = x_train_not_blue - not_blue_average
print(not_blue_minus_mean.shape)

not_blue_cov = np.dot(not_blue_minus_mean, not_blue_minus_mean.T)/ N_not_blue
print(not_blue_cov)


(3, 347713)
[[3219.87199431 1866.08337731  573.04527479]
 [1866.08337731 1481.42775646  753.69162267]
 [ 573.04527479  753.69162267  875.6160161 ]]
(3, 37914963)
[[3834.88244176 3583.18734159 3463.34380916]
 [3583.18734159 3548.6991964  3496.48462157]
 [3463.34380916 3496.48462157 3687.20627384]]


#### Compute the prior probabilities of each class. Again, this is the MLE estimate for the priors.

In [92]:
blue_prior = N_blue / N
print(blue_prior)

not_blue_prior = N_not_blue / N
print(not_blue_prior)

0.009055026041666667
0.987368828125


#### Implementation of multivariate gaussian where covariance inverse and determinant are passed as arguments

In [95]:
def multi_gauss(x, mean, cov_inv, cov_det, dim):
    '''
    Takes an array of one input and calculates its probability based on the Guassians parameters
    
    Arguments:
    x(ndarray): single input to be plugged into pdf
    mean(ndarray): mean of the Gaussian
    cov(ndarray): covariance of the Gaussian
    dim(int): dimension of the Gaussian
    
    Returns:
    probs(ndarray): probabilities for each input
    '''
    
    assert isinstance(x, np.ndarray), "x is not a numpy array"
    assert isinstance(mean, np.ndarray), "mean is not a numpy array"
    assert isinstance(cov_inv, np.ndarray), "cov is not a numpy array"
    assert isinstance(dim, int), "dim is not a int"
    assert x.shape == (3,1)
    assert mean.shape == (3,1)
    assert cov_inv.shape == (3,3)
    
    
    a = 1/np.sqrt( ((2*np.pi)**dim) * cov_det)
    
    minus_mean = x - mean
    exp_term = -0.5*np.dot( np.dot(minus_mean.T, cov_inv), minus_mean)
    exp = np.exp(exp_term)
    
    probs = a*exp
    
    return probs[0][0]


#### Run the barrel detector on both training an validation images. Validation: 1-5, Training: 6-46

In [107]:
import importlib

from hw1_starter_code import barrel_detector
importlib.reload(barrel_detector)

#initialize barrel detector
my_barrel_d = barrel_detector.BarrelDetector()

img = cv2.imread("trainset/1.png")
#obtain the segmentation mask
my_mask = my_barrel_d.segment_image(img)
#obtain a list of bboxes
rects = my_barrel_d.get_bounding_box(img)

#display original image and segmented mask
cv2.imshow('image',img)
cv2.imshow('mask',my_mask*255)
cv2.waitKey(0)
cv2.destroyAllWindows()


#### The following cells were used to test different post-segmentation processing techniques for removing non-barrels.

In [38]:
#post segmentation processing
kernel = np.ones((5,5),np.uint8)
my_mask = cv2.morphologyEx(my_mask,cv2.MORPH_OPEN, kernel)
my_mask = cv2.morphologyEx(my_mask, cv2.MORPH_CLOSE, kernel)

my_mask = cv2.dilate(my_mask,kernel,iterations = 10)
my_mask = cv2.erode(my_mask,kernel,iterations = 10)

        
cv2.imshow('mask',my_mask*255)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [39]:
#remove shapes with small area
img2,cc, hierarchy = cv2.findContours(my_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)      
for c in cc:
    if cv2.contourArea(c) < 500:
        my_mask = cv2.fillPoly(my_mask, pts = [c], color=(0,0,0))

cv2.imshow('mask',my_mask*255)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
#check aspect ratio
img2,cc, hierarchy = cv2.findContours(my_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  
for c in cc:       
    x,y,w,h = cv2.boundingRect(c)
    aspect_ratio_1 = float(w)/h
    aspect_ratio_2 = float(h)/w
    if aspect_ratio_1 > 1 or aspect_ratio_2 > 3:
        my_mask = cv2.fillPoly(my_mask, pts =[c], color=(0,0,0))
        
cv2.imshow('mask',my_mask*255)
cv2.waitKey(0)
cv2.destroyAllWindows()       

In [33]:
#make shapes convex
img2,cc, hierarchy = cv2.findContours(my_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)        
for c in cc:       
    hull = cv2.convexHull(c)
    my_mask = cv2.fillPoly(my_mask, pts =[hull], color=(255,255,255))

cv2.imshow('mask',my_mask*255)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Display the image with bbox(s)

In [108]:
for rect in rects:
    img = cv2.rectangle(img, (rect[0],rect[1]), (rect[2], rect[3]), (0,0,255), 3)
      
cv2.imshow('bboxes',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Probability of error calculation

In [110]:
img_ground_truth = cv2.imread("trainset/1bb.png")
img_ground_truth = np.array(img_ground_truth)  

blue_count = 0
not_blue_count = 0

blue_error_count = 0
not_blue_error_count = 0

for i in range(img_ground_truth.shape[0]):
    for j in range(img_ground_truth.shape[1]):
        if img_ground_truth[i,j,0] == 0:
            not_blue_count+=1
            if my_mask[i,j] == 1:
                not_blue_error_count+=1
        else:
            blue_count+=1
            if my_mask[i,j] == 0:
                blue_error_count+=1

poe = (blue_error_count/blue_count)*blue_prior + (not_blue_error_count/not_blue_count)*not_blue_prior

print(poe)


0.004949993726815879
